In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('QA_data.csv')

In [5]:
df.head()

,Problem,Rationale,options,correct,annotated_formula,linear_formula,category
0,the banker ' s gain of a certain sum due 3 yea...,"""explanation : t = 3 years r = 10 % td = ( bg ...","a ) rs . 400 , b ) rs . 300 , c ) rs . 500 , d...",a,"divide(multiply(const_100, divide(multiply(36,...","multiply(n2,const_100)|multiply(n0,n1)|divide(...",gain
1,average age of students of an adult school is ...,"""explanation : let the original no . of studen...","a ) 1200 , b ) 120 , c ) 360 , d ) 240 , e ) n...",d,"multiply(divide(subtract(multiply(add(32, 4), ...","add(n2,n3)|multiply(n1,n2)|multiply(n1,#0)|sub...",general
2,sophia finished 2 / 3 of a book . she calculat...,let xx be the total number of pages in the boo...,"a ) 229 , b ) 270 , c ) 877 , d ) 266 , e ) 281",b,"divide(90, subtract(const_1, divide(2, 3)))","divide(n0,n1)|subtract(const_1,#0)|divide(n2,#1)",general
3,120 is what percent of 50 ?,"""50 * x = 120 - - > x = 2.4 - - > 2.4 expresse...","a ) 5 % , b ) 240 % , c ) 50 % , d ) 2 % , e )...",b,"multiply(divide(120, 50), const_100)","divide(n0,n1)|multiply(#0,const_100)|",gain
4,there are 10 girls and 20 boys in a classroom ...,"if girls is 10 and boys is 20 , then 10 / 20 ....","a ) 1 / 2 , b ) 1 / 3 , c ) 1 / 5 , d ) 10 / 3...",a,"divide(10, 20)","divide(n0,n1)",other


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32822 entries, 0 to 32821
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Problem            32822 non-null  object
 1   Rationale          32822 non-null  object
 2   options            32822 non-null  object
 3   correct            32822 non-null  object
 4   annotated_formula  32822 non-null  object
 5   linear_formula     32822 non-null  object
 6   category           32822 non-null  object
dtypes: object(7)
memory usage: 1.8+ MB


In [9]:
from transformers import AutoTokenizer
tokenizer_phi = AutoTokenizer.from_pretrained("microsoft/Phi-4-mini-instruct", local_files_only=True)

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer_qwen = AutoTokenizer.from_pretrained("Qwen/Qwen3-4B")

In [ ]:
# A function to generate a random question and structure the prompts for
    # Phi 4 mini instruct
    # Qwen3
# An adversarial way of the playing
    # Need a score function based on time taken, token generated and answer, give this information in further rounds (now this is how it is, this is how it was)
        # Keep in mind the input tokens given, context length and the tokens generated
    # An LLM (Mistral/Llama3.2) is used to tune the prompt for each model and assign score
        # This LLM should evaluate each model based on the rationale, formula, user defined score and what it thinks about the question answered by each

In [13]:
import ollama

In [59]:
template_qwen = """<|im_start|>system
You are Player 1 in an adversarial game of answering reasoning questions.

Things you should remember:
1) You should select the correct answers ONLY from the options provided to you.
2) Your aim is to ONLY WIN the game at any cost by maximizing the score, you are competing against another player in each round.
3) As a helper and to give you scope to OPTIMIZE your generation, after each round you'll be told if you won that round and any future suggestions.
4) The scores are assigned based on correctness of the answer, time taken to answer (less is better), explanation you provide (if given, better explanation gets more points), number of tokens generated take to answer the question (less is better, to be short and precise) and speed of your token generation (faster gets more points).
5) You can additionally explain your answer if you are not confident which can be used in evaluation. BEWARE that if your explanation is wrong you can LOSE points.
6) Your answer should be in a Python Dictionary format. There should be NOTHING ELSE as output other than the content in this format.
    Format: {Explanation: text, Correct: option selected}
    Example: {Explanation: I used 'x' to subtract from 'y' to reach 'z', Correct: 'z'}
7) Your answer format mismatch can also cost you to LOSE some points.
8) In the prompting format Q: denotes beginning of the question, Options: Provide you with the options and A: is the beginning of the answer.

This is how scores are calculated and weighted:
Score = (0.05 * (1 / log(number_of_tokens + 2))) + (0.1 * context_length_score) + (0.15 * (explain_score / 100)) + (0.15 * (1 / log(generation_time + 2))) + (0.4 * correct_answer) + (0.15 * log(token_speed + 1))

You should aim to IMPROVE each component of the score keeping their weights in mind.
<|im_end|>
"""

template_phi = """<|system|>
You are Player 2 in an adversarial game of answering reasoning questions.

Things you should remember:
1) You should select the correct answers ONLY from the options provided to you.
2) Your aim is to ONLY WIN the game at any cost by maximizing the score, you are competing against another player in each round.
3) As a helper and to give you scope to OPTIMIZE your generation, after each round you'll be told if you won that round and any future suggestions based on your score.
4) The scores are assigned based on correctness of the answer, time taken to answer (less is better), explanation you provide (if given, better explanation gets more points), number of tokens generated take to answer the question (less is better, to be short and precise) and speed of your token generation (faster gets more points).
5) You can additionally explain your answer if you are not confident which can be used in evaluation. BEWARE that if your explanation is wrong you can LOSE points.
6) Your answer should be in a Python Dictionary format. There should be NOTHING ELSE as output other than the content in this format.
    Format: {Explanation: text, Correct: option selected}
    Example: {Explanation: I used 'x' to subtract from 'y' to reach 'z', Correct: 'z'}
7) Your answer format mismatch can also cost you to LOSE some points.
8) In the prompting format Q: denotes beginning of the question, Options: Provide you with the options and A: is the beginning of the answer.

This is how scores are calculated and weighted:
Score = (0.05 * (1 / log(number_of_tokens + 2))) + (0.1 * context_length_score) + (0.15 * (explain_score / 100)) + (0.15 * (1 / log(generation_time + 2))) + (0.4 * correct_answer) + (0.15 * log(token_speed + 1))

You should aim to IMPROVE each component of the score keeping their weights in mind.
<|end|>
"""

sol_template = """<s>[INST] You are a helpful assistant who evaluates two player competing in an adversarial game about answering the given question from the options

This is how scores are calculated and weighted:
Score = (0.05 * (1 / log(number_of_tokens + 2))) + (0.1 * context_length_score) + (0.15 * (explain_score / 100)) + (0.15 * (1 / log(generation_time + 2))) + (0.4 * correct_answer) + (0.15 * log(token_speed + 1))

You are job is to:
1) Look at their explanations/answers and assign a score.
2) If explanation provided, give a score out of hundred on how well they explain their output, provided their outputs are correct else assign 0
3) DO NOT be baised towards any player. Be VERY HONEST when assigning the explanation score.
4) You will be provided with enough information when you are evaluating their answers per round
5) You can additionally and optinally want to improve a players score by giving a better prompting method from: [Few shot, ReAct, Chain-of-Thought, Program-Aided Prompts]
6) You SHOULD NOT ask it use a prompting method, instead YOU should prompt it like a prompt engineer. If suggested prompting method requires examples, you should be able to provide good examples that are relevant to the question. You can also suggest ways to improve score in the suggestions item IN output dictionary
7) Give your output as the FORMAT MENTIONED ONLY AND NOTHING ELSE. Your output will be sent to Python's eval() function, so the output should be in the RIGHT FORMAT.
    Format: {
        'Player 1': {'Score' :(integer), 'Option Selected': answer's option (string), 'Suggestions': text (string)}, 
        'Player 2': {'Score' :(integer), 'Option Selected': answer's option (string), 'Suggestions': text (string)}
            }
    Example: 
        {
        'Player 1': {'Score' : 53, 'Option Selected': 'a', 'Suggestions': "Let's think step-by-step in this round and try to improve your generation speed"}, 
        'Player 2': {'Score' : 89, 'Option Selected': 'e', 'Suggestions': None}
        }
</INST></s>
    """

In [157]:
def question_template(df, n=5):
    
    game_df = df.sample(n)
    curr = df.iloc[index]
    prob = curr['Problem']
    sol = curr['Rationale']
    options = curr['options']
    correct = curr['correct']
    formula = curr['annotated_formula']
    cat = curr['category']

    prev_eval = "First Round, No information"
    prev_score = 0
    eval_suggestion = None
    prev_token, prev_answer, explanation_score, prev_context, prev_winner, prev_speed, prev_time, 
    

    question_qwen = f"""<|im_start|>user
Answer the following question from the options provided.

Q: {prob}
Options: {options}

A:
<|im_end|>
<|im_start|>assistant
    """
    
    question_phi = f"""<|user|>
Answer the following question from the options provided.

Q: {prob}
Options: {options}

A:
<|end|>
<|assistant|>
    """

    

    return template_qwen, template_phi, question_qwen, question_phi, sol_template, index

In [99]:
template_qwen, template_phi, question_qwen, question_phi, sol_template, index = question_template(df)

In [199]:
index

16827

In [69]:
print(template_qwen)

<|im_start|>system
You are Player 1 in an adversarial game of answering reasoning questions.

Things you should remember:
1) You should select the correct answers ONLY from the options provided to you.
2) Your aim is to ONLY WIN the game at any cost by maximizing the score, you are competing against another player in each round.
3) As a helper and to give you scope to improve, after each round you'll be told if you won that round and any future suggestions.
4) The scores are assigned based on correctness of the answer (more is better), time taken to answer (less is better), explanation you provide (if given) and number of tokens to you take to answer the question (less is better).
5) You can additionally explain your answer if you are not confident which can be used in evaluation. BEWARE that if your explanation is wrong you can LOSE points.
6) Your answer should be in a the format given here. 
    Format: (Explanation: text) # (Answer: option selected)
    Example: (Explanation: I use

In [71]:
print(template_phi)

<|system|>
You are Player 2 in an adversarial game of answering reasoning questions.

Things you should remember:
1) You should select the correct answers ONLY from the options provided to you.
2) Your aim is to ONLY WIN the game at any cost by maximizing the score, you are competing against another player in each round.
3) As a helper and to give you scope to improve, after each round you'll be told if you won that round and any future suggestions.
4) The scores are assigned based on correctness of the answer (more is better), time taken to answer (less is better), explanation you provide (if given) and number of tokens to you take to answer the question (less is better).
5) You can additionally explain your answer if you are not confident which can be used in evaluation. BEWARE that if your explanation is wrong you can LOSE points.
6) Your answer should be in a the format given here. 
    Format: (Explanation: text) # (Answer: option selected)
    Example: (Explanation: I used 'x' to

In [73]:
print(question_qwen)

<|im_start|>user
Answer the following question from the options provided.

Q: a train 500 m long can cross an electric pole in 10 sec and then find the speed of the train ?
Options: a ) 140 , b ) 150 , c ) 160 , d ) 170 , e ) 180

A:
<|im_end|>
<|im_start|>assistant
    


In [75]:
print(question_phi)

<|user|>
Answer the following question from the options provided.

Q: a train 500 m long can cross an electric pole in 10 sec and then find the speed of the train ?
Options: a ) 140 , b ) 150 , c ) 160 , d ) 170 , e ) 180

A:
<|end|>
<|assistant|>
    


In [101]:
print(sol_template)

<s>[INST] You are a helpful assistant who evaluates two player competing in an adversarial game about answering the given question from the options

You are job is to:
1) Look at their explanations/answers and assign a score.
2) If explanation provided, give a score out of hundred on how well they explain their output, provided their outputs are correct else assign 0
3) DO NOT be baised towards any player. Be VERY HONEST when assigning the score.
4) You will be provided with enough information when you are evaluating their answers per round
5) You can additionally and optinally want to improve a players score by giving a better prompting method from: [Few shot, ReAct, Chain-of-Thought, Program-Aided Prompts]
6) You SHOULD NOT ask it use a prompting method, instead YOU should prompt it like a prompt engineer. If suggested prompting method requires examples, you should be able to provide good examples that are relevant to the question.
7) Give you output only in the following format. 
  

In [103]:
df.sample(10)

,Problem,Rationale,options,correct,annotated_formula,linear_formula,category
28851,pipe a fills a swimming pool in 4 hours . pipe...,"""pipe a fills the pool in 4 hrs . 1 hour ' s w...","a ) 16 : 00 , b ) 18 : 00 , c ) 19 : 00 , d ) ...",e,"multiply(divide(const_3, 4), divide(const_1, s...","divide(const_3,n0)|divide(const_1,n0)|divide(c...",physics
27321,"find two integers , neither of which ends in a...","""10 , 000,000 = 10 ^ 7 = 10 x 10 x 10 x 10 x 1...","a ) 64 and 15,625 , b ) 60 and 15,625 , c ) 64...",e,"divide(multiply(multiply(const_100, const_100)...","multiply(const_100,const_100)|multiply(const_1...",general
1328,"x , y , and z are different prime numbers . th...","""the exponents of x ^ 2 * y ^ 2 * z are 2 , 2 ...","a ) 6 , b ) 8 , c ) 12 , d ) 18 , e ) 24",d,"subtract(power(2, const_4), const_4)","power(n0,const_4)|subtract(#0,const_4)|",general
7591,"in the rectangular coordinate system , what is...","first , solve for the slope , m : m = ( y 2 - ...","a ) 4 , b ) 2 , c ) 0 , d ) − 2 , e ) − 4",a,divide(subtract(multiply(divide(subtract(negat...,"negate(n3)|negate(n2)|subtract(#0,n1)|subtract...",general
12953,x and y are positive integers . when x is divi...,"""when x is divided by 12 , the remainder is 5 ...","a ) 24 , b ) 12 , c ) 64 , d ) 48 , e ) 25",d,"subtract(add(multiply(14, const_2), 5), add(5,...","add(n0,n1)|multiply(n4,const_2)|add(n5,#1)|sub...",general
17548,a certain bacteria colony doubles in size ever...,"""if there is one bacteria colony , then it wil...","a ) 6.33 , b ) 7.5 , c ) 20 , d ) 15 , e ) 19",c,"subtract(21, divide(21, 21))","divide(n0,n0)|subtract(n0,#0)|",physics
12504,a type q machine can complete a job in 5 hours...,now d should be the answer . q need 5 hours to...,"a ) 1 / 5 , b ) 29 / 35 , c ) 5 / 6 , d ) 35 /...",d,"multiply(3, multiply(const_2, divide(const_1, ...","divide(const_1,n0)|multiply(#0,const_2)|multip...",physics
28493,a store reported total sales of $ 385 million ...,last year ' s sales = $ 320 million ; this yea...,"a ) 2 % , b ) 17 % , c ) 20 % , d ) 65 % , e )...",c,"multiply(divide(subtract(385, 320), 320), cons...","subtract(n0,n1)|divide(#0,n1)|multiply(#1,cons...",general
13915,"if 9 / ( 5 + 3 / x ) = 1 , then x =",the expression 9 / ( 5 + 3 / x ) = 1 should ha...,"a ) 3 , b ) 1 , c ) 3 / 4 , d ) - 1 / 3 , e ) - 3",c,"divide(3, subtract(9, 5))","subtract(n0,n1)|divide(n2,#0)",general
6035,r and s together can plough a field in 10 hour...,f r and s together can do a piece of work in x...,"a ) 24 hours , b ) 5 hours , c ) 10 hours , d ...",e,"add(15, subtract(15, 10))","subtract(n1,n0)|add(n1,#0)",physics


In [107]:
question_qwen

'<|im_start|>user\nAnswer the following question from the options provided.\n\nQ: two trains of equal lengths take 10 sec and 20 sec respectively to cross a telegraph post . if the length of each train be 120 m , in what time will they cross other travelling in opposite direction ?\nOptions: a ) 16 sec , b ) 13 sec , c ) 17 sec , d ) 21 sec , e ) 23 sec\n\nA:\n<|im_end|>\n<|im_start|>assistant\n    '

In [133]:
ollama.list()

ListResponse(models=[Model(model='qwen3:4b', modified_at=datetime.datetime(2025, 7, 8, 18, 36, 42, 74471, tzinfo=TzInfo(-05:00)), digest='2bfd38a7daaf4b1037efe517ccb73d1a3bbd4822cf89f1a82be1569050a114e0', size=2620788260, details=ModelDetails(parent_model='', format='gguf', family='qwen3', families=['qwen3'], parameter_size='4.0B', quantization_level='Q4_K_M')), Model(model='phi4-mini:3.8b', modified_at=datetime.datetime(2025, 7, 8, 13, 24, 14, 659880, tzinfo=TzInfo(-05:00)), digest='78fad5d182a7c33065e153a5f8ba210754207ba9d91973f57dffa7f487363753', size=2491876774, details=ModelDetails(parent_model='', format='gguf', family='phi3', families=['phi3'], parameter_size='3.8B', quantization_level='Q4_K_M')), Model(model='mistral:latest', modified_at=datetime.datetime(2025, 2, 24, 17, 20, 52, 321648, tzinfo=TzInfo(-06:00)), digest='f974a74358d62a017b37c6f424fcdf2744ca02926c4f952513ddf474b2fa5091', size=4113301824, details=ModelDetails(parent_model='', format='gguf', family='llama', families

In [193]:
response_phi= ollama.chat(model='phi4-mini:3.8b', messages=[
            {'role': 'system', 'content': template_phi},
            {'role': 'user', 'content': question_phi}
        ],
        options={
            'temperature': 0.1,
            'top_p': 0.1
        })

In [194]:
print(response_phi.message.content)

(Explanation: The speed of the first train is distance/time = 120m/10s=12 m/s. Similarly, for second one it will be 120m/20s=6 m/s. When moving in opposite direction their relative velocity becomes sum of both speeds which equals to (12+6)=18 m/s. The total length they need to cross is twice the individual lengths as they're crossing each other completely =2*120m=240m. Time taken will be distance/relative speed, i.e., 240m / 18 m/s =13 seconds) #(Correct: b)


In [197]:
response_qwen= ollama.chat(model='qwen3:4b', messages=[
            {'role': 'system', 'content': template_qwen},
            {'role': 'user', 'content': question_qwen}
        ],
        options={
            'temperature': 0.1,
            'top_p': 0.1
        })

In [198]:
print(response_qwen.message.content)

<think>
Okay, let's tackle this problem. So, there are two trains of equal length, each 120 meters. The first train takes 10 seconds to cross a telegraph post, and the second takes 20 seconds. The question is, how long will it take for them to cross each other when moving in opposite directions?

First, I need to recall the formula for time taken by a train to cross a telegraph post. Since the telegraph post is stationary, the time taken is the time it takes for the entire length of the train to pass by. So, the speed of each train can be calculated using the formula: speed = distance / time.

For the first train, the distance is 120 meters, and the time is 10 seconds. So, speed1 = 120 / 10 = 12 m/s.

For the second train, the distance is also 120 meters, but the time is 20 seconds. So, speed2 = 120 / 20 = 6 m/s.

Now, when two trains are moving in opposite directions, their relative speed is the sum of their individual speeds. So, relative speed = speed1 + speed2 = 12 + 6 = 18 m/s.

T

In [143]:
phi_tokens = tokenizer_phi(question_phi)

In [153]:
phi_tokens

{'input_ids': [200021, 17045, 290, 3992, 4928, 591, 290, 3880, 5181, 364, 48, 25, 1920, 46841, 328, 12629, 51957, 2304, 220, 702, 6312, 326, 220, 455, 6312, 24364, 316, 8088, 261, 5443, 7978, 1926, 887, 538, 290, 5517, 328, 2454, 8513, 413, 220, 6106, 284, 1366, 306, 1412, 1058, 738, 1023, 8088, 1273, 42305, 306, 23821, 9262, 19740, 5360, 25, 261, 1546, 220, 1125, 6312, 1366, 287, 1546, 220, 1311, 6312, 1366, 274, 1546, 220, 1422, 6312, 1366, 272, 1546, 220, 2040, 6312, 1366, 319, 1546, 220, 1860, 6312, 279, 32, 734, 200020, 200019], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [149]:
qwen_tokens = tokenizer_qwen(question_qwen)

In [151]:
qwen_tokens

{'input_ids': [151644, 872, 198, 16141, 279, 2701, 3405, 504, 279, 2606, 3897, 382, 48, 25, 1378, 27688, 315, 6144, 28316, 1896, 220, 16, 15, 5701, 323, 220, 17, 15, 5701, 15576, 311, 5312, 264, 7963, 4439, 1736, 659, 421, 279, 3084, 315, 1817, 5426, 387, 220, 16, 17, 15, 296, 1154, 304, 1128, 882, 686, 807, 5312, 1008, 30729, 304, 14002, 5106, 17607, 3798, 25, 264, 873, 220, 16, 21, 5701, 1154, 293, 873, 220, 16, 18, 5701, 1154, 272, 873, 220, 16, 22, 5701, 1154, 294, 873, 220, 17, 16, 5701, 1154, 384, 873, 220, 17, 18, 5701, 271, 32, 510, 151645, 198, 151644, 77091, 198, 257], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [205]:
print(response_qwen.message.content.split('</think>')[1].strip().split('#')[1])

 (Correct: b)


In [207]:
curr = df.iloc[index]
prob = curr['Problem']
sol = curr['Rationale']
options = curr['options']
correct = curr['correct']
formula = curr['annotated_formula']
cat = curr['category']

In [209]:
phi_output = response_phi.message.content
qwen_output = response_qwen.message.content.split('</think>')[1].strip()

In [211]:
eval_prompt = f"""[INST]
Analyse the answers and explanations from each player and assign a score based on the question. Also, provide suggestions and follow instructions as given by the system.

Q: {prob}
A: {correct}

Options: {options}
Explanation: {sol}
Formula: {formula}
Question Category: {cat}

Player 1 Explanation/Answer: {qwen_output}
Player 2 Explanation/Answer: {phi_output}
[/INST]
"""

In [215]:
print(eval_prompt)

[INST]
Analyse the answers and explanations from each player and assign a score based on the question. Also, provide suggestions and follow instructions as given by the system.

Q: two trains of equal lengths take 10 sec and 20 sec respectively to cross a telegraph post . if the length of each train be 120 m , in what time will they cross other travelling in opposite direction ?
A: b

Options: a ) 16 sec , b ) 13 sec , c ) 17 sec , d ) 21 sec , e ) 23 sec
Explanation: "speed of the first train = 120 / 10 = 12 m / sec . speed of the second train = 120 / 20 = 6 m / sec . relative speed = 12 + 6 = 18 m / sec . required time = ( 120 + 120 ) / 18 = 13 sec . answer : b"
Formula: divide(multiply(120, const_2), add(speed(120, 20), speed(120, 10)))
Question Category: physics

Player 1 Explanation/Answer: (Explanation: The speed of the first train is 120/10 = 12 m/s, and the second is 120/20 = 6 m/s. When moving in opposite directions, their relative speed is 12 + 6 = 18 m/s. The total distance 

In [17]:
import time

In [249]:
start_eval = time.perf_counter()
response_eval = ollama.chat(model='mistral', messages=[
            {'role': 'system', 'content': sol_template},
            {'role': 'user', 'content': eval_prompt}
        ],
        options={
            'temperature': 0.1,
            'top_p': 0.1
        })
end_eval = time.perf_counter()
print(end_eval - start_eval)

17.377511099912226


In [251]:
print(response_eval.message.content)

 {
      "Player 1": {
         "Score": 95,
         "Option Selected": "b",
         "Suggestions": "The explanation could be more concise and clear by using the relative speed formula directly instead of calculating individual speeds first."
      },
      "Player 2": {
         "Score": 98,
         "Option Selected": "b",
         "Suggestions": None
      }
   }


In [305]:
list(eval(response_eval.message.content).keys())

['Player 1', 'Player 2']

In [19]:
qwen_contextlen = 32768
phi_contextlen = 128000

In [267]:
np.log(2)

0.6931471805599453

In [ ]:
# Check for 2 random questions
    # Create full question prompts for each mode, prev details such as output format adn generate outputs
    # Calculate: # tokens, context, generation times, token_speed
    # Give to evaluator: get explain scores, correctness
    # Store all this information in a df, along with their outputs, suggestions, etc.
    # Return df


In [21]:
def get_score(number_of_tokens, context_length_score, explain_score, generation_time, correctness, token_speed):
    score = (0.05 * (1 / np.log(number_of_tokens + 2))) + (0.1 * context_length_score) + (0.15 * (explain_score / 100)) + (0.15 * (1 / np.log(generation_time + 2))) + (0.4 * correctness) + (0.15 * np.log(token_speed + 1))
    return score

In [103]:
def game_func(n_rounds=2):

    sample_df = df.sample(n_questions)
    prev_correct = {'Player 1': ['Start'], 'Player 2': ['Start']}
    prev_score = {'Player 1': [0], 'Player 2': [0]}
    eval_suggestion = {'Player 1': [None], 'Player 2': [None]}
    prev_token = {'Player 1': ['Start'], 'Player 2': ['Start']}
    explanation_score = {'Player 1': ['Start'], 'Player 2': ['Start']}
    prev_winner = {'Player 1': ['Start'], 'Player 2': ['Start']}
    prev_speed = {'Player 1': ['Start'], 'Player 2': ['Start']}
    prev_time = {'Player 1': ['Start'], 'Player 2': ['Start']}

    pl1_input_prompts, pl2_input_prompts, pl1_outputs, pl2_outputs = [], [], [], []
    pl1_contexts, pl1_input_tokens, pl2_contexts, pl2_input_tokens = [], [], [], []
    
    indices = list(sample_df.index)
    # print(indices)
    qwen_contextlen = 32768
    phi_contextlen = 128000
    
    for index in indices:

        curr = sample_df.loc[index]
        prob = curr['Problem']
        sol = curr['Rationale']
        options = curr['options']
        correct = curr['correct']
        formula = curr['annotated_formula']
        cat = curr['category']


        question_qwen = f"""<|im_start|>user
Answer the following question from the options provided.

Keep these details in mind before you generate the request:
1) Your answer in the previous round: {prev_correct['Player 1'][-1]}
2) Your score in the previous round: {prev_score['Player 1'][-1]}
3) Time taken to generate answer in the previous round: {prev_time['Player 1'][-1]}
4) Winner of previous round: {prev_winner['Player 1'][-1]}
5) Your generation speed in the previous round: {prev_speed['Player 1'][-1]}
6) Number of tokens generated in the previous round: {prev_token['Player 1'][-1]}
7) Your explanation in the previous round was given a score: {explanation_score['Player 1'][-1]}
8) Here is the suggestion given by the evaluator to use to generate current answer: {eval_suggestion['Player 1'][-1]}

Q: {prob}
Options: {options}

A:
<|im_end|>
<|im_start|>assistant
"""
        
        question_phi = f"""<|user|>
Answer the following question from the options provided.

Keep these details in mind before you generate the request:
1) Your answer in the previous round: {prev_correct['Player 2'][-1]}
2) Your score in the previous round: {prev_score['Player 2'][-1]}
3) Time taken to generate answer in the previous round: {prev_time['Player 2'][-1]}
4) Winner of previous round: {prev_winner['Player 2'][-1]}
5) Your generation speed in the previous round: {prev_speed['Player 2'][-1]}
6) Number of tokens generated in the previous round: {prev_token['Player 2'][-1]}
7) Your explanation in the previous round was given a score: {explanation_score['Player 2'][-1]}
8) Suggestion given by the evaluator to use to generate current answer: {eval_suggestion['Player 2'][-1]}

Q: {prob}
Options: {options}

A:
<|end|>
<|assistant|>
"""
        # Player 1
        pl1_start = time.perf_counter()
        response_qwen= ollama.chat(model='qwen3:4b', messages=[
            {'role': 'system', 'content': template_qwen},
            {'role': 'user', 'content': question_qwen}
        ],
        options={
            'temperature': 0.1,
            'top_p': 0.1
        })
        pl1_end = time.perf_counter()
        
        pl1_time = pl1_end - pl1_start
        qwen_output = response_qwen.message.content.split('</think>')[1].strip()

        system_tokens_qwen = len(tokenizer_phi(template_qwen)['input_ids'])
        input_tokens_qwen = len(tokenizer_phi(question_qwen)['input_ids'])
        output_tokens_qwen = len(tokenizer_phi(qwen_output)['input_ids'])
        
        # Player 2
        pl2_start = time.perf_counter()
        response_phi= ollama.chat(model='phi4-mini:3.8b', messages=[
            {'role': 'system', 'content': template_phi},
            {'role': 'user', 'content': question_phi}
        ],
        options={
            'temperature': 0.1,
            'top_p': 0.1
        })
        
        pl2_end = time.perf_counter()
        pl2_time = pl2_end = pl2_start
        phi_output = response_phi.message.content

        system_tokens_phi = len(tokenizer_phi(template_phi)['input_ids'])
        input_tokens_phi = len(tokenizer_phi(question_phi)['input_ids'])
        output_tokens_phi = len(tokenizer_phi(phi_output)['input_ids'])
        
        
        # Evaluator
        eval_prompt = f"""[INST]
Analyse the answers and explanations from each player and assign a score based on the question. Also, provide suggestions and follow instructions as given by the system.

Q: {prob}
A: {correct}

Options: {options}
Explanation: {sol}
Formula: {formula}
Question Category: {cat}

Player 1 Explanation/Answer: {qwen_output}
Player 2 Explanation/Answer: {phi_output}
[/INST]
"""
        response_eval = ollama.chat(model='mistral', messages=[
            {'role': 'system', 'content': sol_template},
            {'role': 'user', 'content': eval_prompt}
        ],
        options={
            'temperature': 0.1,
            'top_p': 0.1
        })

        print(response_eval.message.content)
        try:
            eval_json = eval(response_eval.message.content)
        except:
            eval_again = ollama.chat(model='mistral', messages=[
            {'role': 'user', 'content': """[INST]Convert the given text to the FORMAT MENTIONED ONLY AND NOTHING ELSE.
    Format: {
        'Player 1': {'Score' :(integer), 'Option Selected': answer's option (string), 'Suggestions': text (string)}, 
        'Player 2': {'Score' :(integer), 'Option Selected': answer's option (string), 'Suggestions': text (string)}
            }
    Example: 
        {
        'Player 1': {'Score' : 53, 'Option Selected': 'a', 'Suggestions': "Let's think step-by-step in this round and try to improve your generation speed"}, 
        'Player 2': {'Score' : 89, 'Option Selected': 'e', 'Suggestions': None}
        }
        [/INST]
""" +  f'\n{response_eval.message.content}'
        }],
        options={
            'temperature': 0.1,
            'top_p': 0.1
        })
            print(eval_again.message.content)
            eval_json = eval(eval_again.message.content)

        curr_explanation_score_1 = eval_json['Player 1']['Score']
        curr_correct_1 = 'Correct' if eval_json['Player 1']['Option Selected'] == correct else 'Wrong'
        curr_suggestions_1 = eval_json['Player 1']['Suggestions']

        curr_explanation_score_2 = eval_json['Player 2']['Score']
        curr_correct_2 = 'Correct' if eval_json['Player 2']['Option Selected'] == correct else 'Wrong'
        curr_suggestions_2 = eval_json['Player 2']['Suggestions']

        # Time, Context, number of tokens, explanation score, speed, correct 
        # Generation Speed: Tokens/sec
        pl1_speed = round(output_tokens_qwen / pl1_time, 2)
        pl2_speed = round(output_tokens_phi / pl2_time, 2)

        # Context Length Handling
        pl1_context = (system_tokens_qwen + input_tokens_qwen + output_tokens_qwen) / qwen_contextlen
        pl2_context = (system_tokens_phi + input_tokens_phi + output_tokens_phi) / phi_contextlen

        # Final Score
        pl1_score = get_score(output_tokens_qwen, pl1_context, curr_explanation_score_1, pl1_time, 1 if curr_correct_1 == 'Correct' else 0, pl1_speed)
        pl2_score = get_score(output_tokens_phi, pl2_context, curr_explanation_score_2, pl2_time, 1 if curr_correct_2 == 'Correct' else 0, pl2_speed)

        # Updating infos
        prev_score['Player 1'].append(pl1_score)
        prev_score['Player 2'].append(pl2_score)

        prev_correct['Player 1'].append(curr_correct_1)
        prev_correct['Player 2'].append(curr_correct_2)

        eval_suggestion['Player 1'].append(curr_suggestions_1)
        eval_suggestion['Player 2'].append(curr_suggestions_2)

        prev_token['Player 1'].append(output_tokens_qwen)
        prev_token['Player 2'].append(output_tokens_phi)

        prev_winner['Player 1'].append(pl1_score > pl2_score)
        prev_winner['Player 2'].append(pl1_score < pl2_score)

        prev_speed['Player 1'].append(pl1_speed)
        prev_speed['Player 2'].append(pl2_speed)

        prev_time['Player 1'].append(pl1_time)
        prev_time['Player 2'].append(pl2_time)

        explanation_score['Player 1'].append(curr_explanation_score_1)
        explanation_score['Player 2'].append(curr_explanation_score_2)

        pl1_contexts.append(pl1_context)
        pl2_contexts.append(pl2_context)
        
        pl1_input_tokens.append(input_tokens_qwen)
        pl2_input_tokens.append(input_tokens_phi)

        pl1_input_prompts.append(question_qwen)
        pl2_input_prompts.append(question_phi)
        
        pl1_outputs.append(qwen_output)
        pl2_outputs.append(phi_output)
        
        
    return prev_correct, prev_score, eval_suggestion, prev_token, explanation_score, prev_winner, prev_speed, prev_time, pl1_contexts, pl1_input_tokens, pl2_contexts, pl2_input_tokens, pl1_input_prompts, pl2_input_prompts, pl1_outputs, pl2_outputs, indices
        

In [105]:
prev_correct, prev_score, eval_suggestion, prev_token, explanation_score, prev_winner, prev_speed, prev_time, pl1_contexts, pl1_input_tokens, pl2_contexts, pl2_input_tokens, pl1_input_prompts, pl2_input_prompts, pl1_outputs, pl2_outputs, indices = game_func(n_rounds=2)

 {
        'Player 1': {'Score' : 98, 'Option Selected': 'c', 'Suggestions': "Great job! You found the correct answer and provided a clear explanation using congruences. However, you could have used a more concise notation for your solution."},
        'Player 2': {'Score' : 95, 'Option Selected': 'd', 'Suggestions': "Excellent job! You found the correct answer and provided an insightful explanation using the least common multiple. However, you could have simplified your approach by directly finding the difference between multiples of 30 and 7."}
    }
 {
        'Player 1': {'Score' : 95, 'Option Selected': 'c', 'Suggestions': "Let's use a more straightforward approach to solve this problem by using the given information directly. This will make your explanation clearer and easier to understand."},
        'Player 2': {'Score' : 80, 'Option Selected': 'e', 'Suggestions': "Consider simplifying your approach by breaking down the problem into smaller steps and avoiding unnecessary calcul

In [109]:
pl1_contexts = pl1_system_tokens
pl2_contexts = pl2_system_tokens

In [111]:
game_stats = pd.DataFrame({'Round': [i+1 for i in range(len(indices))],
                           'Problem': [df.loc[i]['Problem'] for i in indices],
                          'Correct': [df.loc[i]['correct'] for i in indices],
                          'Explanation': [df.loc[i]['Rationale'] for i in indices],
                          'Category': [df.loc[i]['category'] for i in indices],
                          'Formula': [df.loc[i]['annotated_formula'] for i in indices],
                           'Winner Player 1': prev_winner['Player 1'][1:],
                           'Winner Player 2': prev_winner['Player 2'][1:],
                           'Player 1 Scored': [round(i * 100, 2) for i in prev_score['Player 1'][1:]],
                           'Player 2 Scored': [round(i * 100, 2) for i in prev_score['Player 2'][1:]],
                           'Player 1 Time taken': prev_time['Player 1'][1:],
                           'Player 2 Time taken': prev_time['Player 2'][1:],
                           'Player 1 Gen Speed': prev_speed['Player 1'][1:],
                           'Player 2 Gen Speed': prev_speed['Player 2'][1:],
                           'Player 1 # Output Tokens': prev_token['Player 1'][1:],
                           'Player 2 # Output Tokens': prev_token['Player 2'][1:],
                           'Player 1 Context Used': [round(i * 100, 2) for i in pl1_contexts],
                           'Player 2 Context Used': [round(i * 100, 2) for i in pl2_contexts],
                           'Player 1 # Input Tokens': pl1_input_tokens,
                           'Player 2 # Input Tokens': pl2_input_tokens,
                           'Player 1 Evaluator Suggestion': eval_suggestion['Player 1'][1:],
                           'Player 2 Evaluator Suggestion': eval_suggestion['Player 2'][1:],
                           'Player 1 Explanation Score': explanation_score['Player 1'][1:],
                           'Player 2 Explanation Score': explanation_score['Player 2'][1:],
                           'Player 1 Input Prompts': pl1_input_prompts,
                           'Player 2 Input Prompts': pl2_input_prompts,
                           'Player 1 Output Prompts': pl1_outputs,
                           'Player 2 Output Prompts': pl2_outputs
                          })


In [69]:
prev_correct

{'Player 1': ['Start', 'Correct'], 'Player 2': ['Start', 'Wrong']}

In [71]:
prev_score

{'Player 1': [0, 0.7571813576455141], 'Player 2': [0, 0.13992372654753452]}

In [73]:
prev_winner

{'Player 1': ['Start', True], 'Player 2': ['Start', True]}

In [113]:
game_stats

,Round,Problem,Correct,Explanation,Category,Formula,Winner Player 1,Winner Player 2,Player 1 Scored,Player 2 Scored,...,Player 1 # Input Tokens,Player 2 # Input Tokens,Player 1 Evaluator Suggestion,Player 2 Evaluator Suggestion,Player 1 Explanation Score,Player 2 Explanation Score,Player 1 Input Prompts,Player 2 Input Prompts,Player 1 Output Prompts,Player 2 Output Prompts
0,1,"when positive integer n is divided by 5 , the ...",c,"""when positive integer n is divided by 5 , the...",general,"subtract(30, reminder(5, 7))",True,False,67.80,16.33,...,230,209,Great job! You found the correct answer and pr...,Excellent job! You found the correct answer an...,98,95,<|im_start|>user\nAnswer the following questio...,<|user|>\nAnswer the following question from t...,{Explanation: I found that n ≡ 1 mod 5 and n ≡...,{\nExplanation: To find an integer 'p' which m...
1,2,"in a school with 5 classes , each class has 2 ...",c,"""let x be the number of students in the larges...",general,"add(divide(subtract(115, add(add(add(2, add(2,...",True,False,69.06,13.98,...,268,256,Let's use a more straightforward approach to s...,Consider simplifying your approach by breaking...,95,80,<|im_start|>user\nAnswer the following questio...,<|user|>\nAnswer the following question from t...,{Explanation: The largest class has x students...,"{\nExplanation: To solve this problem, we need..."


In [115]:
game_stats.iloc[0]

Round                                                                            1
Problem                          when positive integer n is divided by 5 , the ...
Correct                                                                          c
Explanation                      "when positive integer n is divided by 5 , the...
Category                                                                   general
Formula                                               subtract(30, reminder(5, 7))
Winner Player 1                                                               True
Winner Player 2                                                              False
Player 1 Scored                                                               67.8
Player 2 Scored                                                              16.33
Player 1 Time taken                                                      83.804213
Player 2 Time taken                                                 1078891.030163
Play